In [91]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.tag import StanfordPOSTagger
from nltk.corpus import wordnet
import pandas as pd
from langdetect import detect
from stop_words import get_stop_words
from collections import defaultdict
import pprint
from googletrans import Translator
import re

#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')


In [116]:
stop_words = get_stop_words('fr')
df = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t')
#translator = Translator()
lemmatizer = FrenchLefffLemmatizer()

In [93]:
# translator.translate('안녕하세요.')

In [94]:
%%time

def languagesDetectionDF(column):
    languagesTitle = defaultdict(int)
    for cell in df[column]:
        if type(cell) is str:
            languagesTitle[detect(cell)] += 1
#         if lang != 'fr' and lang != 'en':
#             print(title)
    return languagesTitle

languageTitle = languagesDetectionDF("title")
languageAbstract = languagesDetectionDF("abstract")

CPU times: user 8.52 s, sys: 129 ms, total: 8.65 s
Wall time: 8.65 s


In [95]:
print("Langage des titres :", languageTitle)

print("Langage des abstract :", languageAbstract)

Langage des titres : defaultdict(<class 'int'>, {'fr': 1133, 'en': 124, 'tl': 1, 'it': 1, 'ca': 5, 'de': 2, 'fi': 1, 'da': 1, 'af': 1})
Langage des abstract : defaultdict(<class 'int'>, {'fr': 991, 'en': 105})


In [96]:
invYr = df.groupby(['year']).count()

print(invYr['abstract'].mean())
print(invYr['abstract'].std())
print(invYr['abstract'].min())
print(invYr['abstract'].max())

73.06666666666666
13.884558054938314
56
98


In [97]:
#tagger fr
jar = 'stanford-postagger-full-2018-10-16/stanford-postagger-3.9.2.jar'
model = 'stanford-postagger-full-2018-10-16/models/french.tagger'
import os
java_path = "/usr/bin/java"
os.environ['JAVAHOME'] = java_path

#appel super long
pos_tagger = StanfordPOSTagger(model, jar, encoding='utf8' )

In [98]:
%%time
def wordNetTag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def lemma(tokenizedText):
    taggedText = pos_tagger.tag(tokenizedText)
    res = []
    for word in taggedText:
        tag = wordNetTag(word[1])
        if tag:
            res.append(lemmatizer.lemmatize(word[0], tag))

        else:
            res.append(lemmatizer.lemmatize(word[0]))
    return(res)


CPU times: user 28 µs, sys: 2 µs, total: 30 µs
Wall time: 42.2 µs


In [99]:
%%time
def lemmatizerDF(dfColumn):
    columnLemma = []
    for cell in dfColumn:
        if type(cell) is str:
            if detect(cell) == 'fr':
                filtered_sentence = [w.lower() for w in word_tokenize(cell) if not w in stop_words] 
                columnLemma.append(' '.join(lemma(filtered_sentence)))
            else:
                columnLemma.append(cell)
        else:
            columnLemma.append('')
    return columnLemma
        

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [12]:
%%time

#C'est très long parce que chaque utilisation du tagger prend environ une demie seconde et qu'on l'utilise pour chaque ligne

titleLemma = lemmatizerDF(df["title"])

CPU times: user 7.47 s, sys: 5.73 s, total: 13.2 s
Wall time: 7min 52s


In [13]:
%%time

#C'est très long parce que chaque utilisation du tagger prend environ une demie seconde et qu'on l'utilise pour chaque ligne

abstractLemma = lemmatizerDF(df["abstract"])

CPU times: user 7.52 s, sys: 5.31 s, total: 12.8 s
Wall time: 9min 1s


In [40]:
df["titleLemma"] = titleLemma
df["abstractLemma"] = abstractLemma

NameError: name 'titleLemma' is not defined

In [24]:
df.to_csv('dataLemmatized.csv')